In [ ]:
from nltk.corpus import stopwords
from tqdm import tqdm

import re

import pandas as pd
stop_words = stopwords.words('english')

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
def remove_stuff(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r"@\S*", "", text)
    text = re.sub(r"\s+$", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

In [ ]:
df = pd.read_csv("./mentioner.csv")
df.columns

In [ ]:
usernames = df["user"].unique()

In [ ]:
wordlist = pd.DataFrame(columns=["word", "pos"])
word_counts = dict.fromkeys(usernames)

In [ ]:
docs = []

for i, tweet in enumerate(tqdm(df["text"])):
    tweets = remove_stuff(tweet)
    doc = nlp(tweet)

    user = df["user"][i]

    for token in doc:
        text = token.orth_
        pos = token.pos_

        if pos not in wordlist["word"]:
            wordlist = pd.concat((
                wordlist,
                pd.DataFrame({
                    "word": [text],
                    "pos": [pos]
                })
            ))

    docs.append(doc) 

In [ ]:
df["doc"]=docs

In [ ]:
wordlist.to_csv("./wordlist.csv", index=False)

In [ ]:
wordlist = pd.read_csv("./wordlist.csv")
wordlist

In [ ]:
wordlist.reset_index(drop=True, inplace=True)
wordlist.drop_duplicates(inplace=True)
wordlist.set_index("word", inplace=True)
wordlist

In [ ]:
for user in usernames:
    subset = df[df["username"] == user]
    wordlist[user] = [0] * len(wordlist)

    docs = subset["tweet_content"].map(nlp)

    for doc in tqdm(docs, desc=user):
        for item in doc:
            word = item.orth_
            pos = item.pos_

            wordlist.loc[(wordlist.index == word) & (wordlist["pos"] == pos), user] += 1

In [ ]:
wordlist.reset_index().to_csv("./wordlist.csv", index=False)

In [ ]:
wordlist.to_csv("./wordlist.csv", index=False)